In [3]:
# RetrievalQA

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter 
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings # 분할된 문서와 openAI embeddings model을 전달해야된다.
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model="gpt-3.5-turbo")

cache_dir = LocalFileStore("./.cache/") # .cache 디렉토리를 생성됨 (여기에 임베딩된 데이터들이 저장됨)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", # 문단 기준으로 분할
    chunk_size=600, # 최대 글자개수 600인 문서로 나눔 (separator가 없으면 좀 더 커질수도 있고 작아질 수도 있음)
    chunk_overlap=100,
)

# UnstructuredFileLoader: 다양한 파일 로드가능 (pdf, html, doc, txt, excel 등등)
loader = UnstructuredFileLoader("./files/test.txt")

# 긴 문서 텍스트를 작은 문서들로 나누어 LLM에게 전달.
# 거대한 단일 문서보다는 작은 여러개를 전달할 때 검색 성능이 더 좋아짐. 더 저렴해지고, 작업도 쉬워지고, 응답도 바름
docs = loader.load_and_split(text_splitter=splitter)

# 임베딩은 텍스트에 의미별로 적절한 점수를 부여해 vector형식으로 표현
embeddings = OpenAIEmbeddings()

# CacheBackedEmbeddings을 사용해 만들어진 embedding을 cache(저장)함. 임베딩도 공짜는 아님
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# FAISS를 사용해 Vector Store를 호출 docs와 임베딩을 전달
# 문서별로 임베딩 작업 후 결과를 저장한 Vector Store를 반환하는데, 이걸로 문서 검색 및 연관성이 높은 문서들을 찾기도 함
vector_store = FAISS.from_documents(
    docs,
    # embedding=embeddings,
    cached_embeddings # 처음엔 캐시에 임베딩이 존재하는지 확인. 재 호출시 캐싱되어있는 임베딩을 전달
)

# 리트리버 사용
chain = RetrievalQA.from_chain_type(
    llm=llm,
    # chain_type="stuff",
    chain_type="refine", # refine(정제)
    # refine은 각각의 문서를 읽으면서 질문에 대한 답변을 생성함, foo는 무엇입니까? 하면 개별 문서마다 질문을 하고, 각 문서마다 질문 하면서 더 나은 답변으로 가공함
    # 예를 들어 문서가 4개있다 치고, 첫번째 문서를 읽고 답변을 생성(처음엔 별로 좋지 않은 답변) -> 두번째 문서에를 읽고 거기 있는 정보로 또 답변을 업데이트함 -> 계속 반복해서 더 나은 답변을 제공 -> 당연히 요금이 더 비쌈
    # 문서가 10개 있으면 10번 질문을 함
    # chain_type="map_rerank",
    retriever=vector_store.as_retriever(), # retriever는 랭체인이 제공하는 class 또는 interface일종인데, 문서를 검색해서 찾아오는 기능을 가지고 있음
)
print(chain.run("내용을 문맥에 맞게 잘 요약 해주세요"))

헬렌은 자비로운 마음을 가진 존엄한 존재로 노래하며 트로이 전쟁의 상처와 희생에 대해 회상한다. 그녀의 존엄성과 희생은 전쟁의 끝과 함께 찬양을 받고, 헬렌과 트로이는 이제 별처럼 차가워졌다. 이에 대한 노래와 헬렌의 마음의 휴식을 바라는 간절한 염원이 담긴 시를 외친다. 트로이 전쟁과 헬렌의 이야기는 전쟁의 상처와 희생, 그리고 자비로운 마음에 대한 깊은 사상을 담고 있다.
